In [1]:
import os, sys
import pandas as pd
import numpy as np
import datetime
from datetime import date

In [6]:
def clean_citibike(data):
    data['datetime'] = pd.to_datetime(data['date'], format='%y-%m-%d')
    data.drop('date', axis = 1, inplace = True)
    #binary feature of each day
    data['date'] = data['datetime'].dt.date
    data['day_of_week'] = data.datetime.dt.weekday_name
    data['weekday'] = [0 if x in ['Saturday','Sunday'] else 1 for x in data['day_of_week']]

    #binarize categorical variables
    data = pd.get_dummies(data , columns = ['day_of_week'] )
    
    #holiday    
    holiday = ['2016-01-01','2016-01-18', '2016-02-15', '2016-05-30',
               '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24', '2016-12-25']

    holiday = pd.to_datetime(holiday, format='%Y-%m-%d')
    data['holiday'] = [1 if x in holiday else 0 for x in data.datetime]

    #cyclical characteristics of time
    data['hour_24'] = data.hour + data.pm * 12 
    data['time_x'] = np.cos((data.hour_24 + data.minute/60) * 2*np.pi /24)
    data['time_y'] = np.sin((data.hour_24 + data.minute/60) * 2*np.pi /24)
#     data['dayofyear'] = data.datetime.dt.dayofyear
#     data['day_x'] = np.cos((data.dayofyear) * 2*np.pi /365)
#     data['day_y'] = np.sin((data.dayofyear) * 2*np.pi /365)

    #Target 
    data['shortage'] = [1 if x<=5 else 0 for x in data['avail_bikes']]
    
    #remove unwanted columns
    data.drop(['datetime','in_service','status_key', 'avail_bikes','avail_docks'], axis = 1, inplace = True)
    return data



def parse_weather(file_path):

    weather_data = pd.read_csv(file_path)

    weather = weather_data.loc[:,['pickup_datetime',
                        'tempi', #temperature in farenheit
                        'wspdi', #wind speed in mph
                       'rain', # rain, boolean
                       'snow',# snow, boolean
                       'visi' ]] #vibilitity in miles
    weather_features = list(weather.columns[1:])


    weather.fillna(0, inplace=True)
    weather['datetime'] = pd.to_datetime(weather['pickup_datetime'], format='%Y-%m-%d %H:%M::%S', infer_datetime_format=True)
    weather['date'] = weather['datetime'].dt.date
    weather['hour_24'] = weather['datetime'].dt.hour+1
    weather['minute'] = weather['datetime'].dt.minute
    weather.drop(['pickup_datetime', 'datetime'],axis = 1, inplace = True)

    g = weather[['tempi', 'wspdi', 'rain', 'snow','visi']].groupby([weather['date'], weather['hour_24']])
    weather_by_hour = g.mean()

    for i in ['rain', 'snow']:
        weather_by_hour[i] = [1 if w>0 else 0 for w in weather_by_hour[i]]

    weather_by_hour.reset_index(inplace = True)
    return weather_by_hour

def merge_weather (weather_df, df):

    df['date'] = pd.to_datetime(df['date']).dt.date
    df_weather = pd.merge(df, weather_df, on=['date','hour_24'], how = 'left')
    df_weather.fillna(method = 'bfill', inplace = True)
    return df_weather


In [ ]:
#read data
data_dir = '../Data/'
data_train = pd.read_csv(data_dir + 'train1_jan_aug.csv', sep='\t', error_bad_lines=False, index_col = 'Unnamed: 0')
data_test = pd.read_csv(data_dir + 'test_sep_oct.csv', sep='\t', error_bad_lines=False,  index_col = 'Unnamed: 0')

#remove unreasonable data
    #total docks == 0
data_test = data_test[data_test.tot_docks != 0]
data_train = data_train[data_train.tot_docks != 0]
    #coming soon bike docks
data_train = data_train[~data_train.dock_name.str.contains('Coming Soon')]
data_test = data_test[~data_test.dock_name.str.contains('Coming Soon')]

#clean
d_test = clean_citibike(data_test)
d_train = clean_citibike(data_train)

#clean and merge weather data
weather_file_path = data_dir + 'Weather.csv'
weather_by_hour = parse_weather(weather_file_path)
d_train = merge_weather(weather_by_hour, d_train)
d_test = merge_weather(weather_by_hour,d_test)

#merge subway data
subway = pd.read_csv(data_dir + 'dock_subway_dist.csv')
subway = subway[['dock_name', 'lat','long','dist']]

subway = subway.rename(columns={'lat': 'subway_lat', 'long': 'subway_long'})
subway = subway.loc[subway.groupby('dock_name').subway_lat.idxmin()]

d_train = pd.merge(d_train, subway, how = 'inner', on ='dock_name')
d_test = pd.merge(d_test, subway, how = 'inner', on ='dock_name')


#drop unnecessary columns
d_train = d_train.drop(['hour_24','hour', 'minute', 'pm' ,'date','dock_id','dock_name','dock_name6'],1)
d_test = d_test.drop(['hour_24','hour', 'minute', 'pm' ,'date','dock_id','dock_name'],1)

In [7]:
d_train.to_csv('../Data/train_very_clean.csv', sep = ',', index = False)
d_test.to_csv('../Data/test_very_clean.csv', sep = ',', index = False)